### Libraries and Dependencies



In [ ]:
%%capture
%env TORCH=2.1.0+cu121
!pip install torch_scatter torch_sparse torch_spline_conv -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torchmetrics ogb rdkit
!pip install torch-geometric

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from typing import Tuple

from torch import Tensor
from torch.nn import Module, Dropout, LayerNorm, Identity
from torch_geometric.data import DataLoader, Data, InMemoryDataset
from torch_geometric.nn import GINConv, GINEConv, global_add_pool
from torch_geometric.typing import Adj, OptTensor
from torch_geometric.utils import degree
from torch_geometric.utils.convert import from_networkx
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import networkx as nx
from torch_geometric.nn import GCNConv
import torch_geometric.nn as gnn
from torch.autograd import Variable

from google.colab import drive

/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:72: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:99: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while

In [ ]:

import pickle
# Save the object to a file
file_path = 'transformed_dataset_20.pkl'  # Specify the path and file name

# Opening the file in binary-write mode and saving with pickle
with open(file_path, 'rb') as file:
    transformed_dataset_20 = pickle.load( file)

/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:72: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:99: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while

EOFError: Ran out of input

#DATASETS

In [ ]:
data, slices = torch.load('exp_dataset.pt')

class PlanarSATPairsDataset(InMemoryDataset):
    def __init__(self):
        super(PlanarSATPairsDataset, self).__init__()
        self.data, self.slices = data, slices
exp_dataset = PlanarSATPairsDataset()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
len(exp_dataset)

In [ ]:
def split_dataset(dataset):
    # Extract data and slices from the dataset
    data, slices = dataset.data, dataset.slices

    # Prepare lists to hold separate graph Data objects, features, and labels
    graphs = []
    features_list = []
    labels_list = []

    # Iterating over the range of total graphs (assuming 'y' is always present and is per graph)
    for i in range(len(slices['y']) - 1):
        # Extract indices for node features and edge indices
        x_start, x_end = slices['x'][i], slices['x'][i+1]
        edge_index_start, edge_index_end = slices['edge_index'][i], slices['edge_index'][i+1]
        y = data.y[slices['y'][i]:slices['y'][i+1]]  # y is typically a single label if per graph

        # Create Data object for each graph
        graph = Data(x=data.x[x_start:x_end], edge_index=data.edge_index[:, edge_index_start:edge_index_end], y=y)
        graphs.append(graph)


    return graphs

In [ ]:
def extract_enclosing_subgraph(graph, node_pair, h):
    """Extracts the h-hop enclosing subgraph around a node pair and adds an edge between them."""
    u, v = node_pair
    # Get nodes reachable within h hops from both u and v
    reachable_from_u = set(nx.single_source_shortest_path_length(graph, u, cutoff=h).keys())
    reachable_from_v = set(nx.single_source_shortest_path_length(graph, v, cutoff=h).keys())
    enclosing_nodes = reachable_from_u.union(reachable_from_v)

    # Create subgraph and ensure it includes the edge between u and v
    subgraph = graph.subgraph(enclosing_nodes).copy()
    subgraph.add_edge(u, v)  # Add the edge, which might not exist in the original graph

    return subgraph, (u,v)

def create_line_graph_and_features(subgraph, original_features, node_pair):
    """
    Creates a line graph from a subgraph with new node features being the sum of the
    original edge nodes' features and finds the new index of a node in the line graph
    corresponding to the node_pair.
    """
    # Create the line graph from the subgraph
    line_graph = nx.line_graph(subgraph)

    # Compute features for the line graph nodes
    line_features = {}
    for edge in line_graph.nodes():
        # Sum features of the original nodes that form the edge in the subgraph
        line_features[edge] = original_features[edge[0]] + original_features[edge[1]]

    # Convert the line graph to a PyG Data object
    lg_data = from_networkx(line_graph)

    # Assign features to the line graph in PyG
    lg_features = torch.stack([line_features[edge] for edge in line_graph.nodes()])
    lg_data.x = lg_features

    # Determine the node in the line graph corresponding to the node_pair
    # It's assumed the node_pair is an edge in the original subgraph
    node_mapping = {edge: i for i, edge in enumerate(line_graph.nodes())}
    node_pair = tuple(sorted(node_pair))  # Ensure the pair is sorted (smaller, larger) if undirected
    new_index = node_mapping.get(node_pair, -1)  # -1 if not found

    return lg_data, new_index

In [ ]:
import torch
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx, from_networkx
from itertools import combinations

def transform_graph_to_subgraphs(graph_list, h):
    transformed_data = []

    for data in graph_list:
        G = to_networkx(data, to_undirected=True)
        num_nodes = data.num_nodes
        original_features = {node: data.x[i] for i, node in enumerate(G.nodes())}

        line_graphs = []

        y = data.y

        # Iterate over all pairs of nodes
        for node_pair in combinations(range(num_nodes), 2):
            # Extract the h-hop enclosing subgraph around these two nodes and the edge
            subgraph, _ = extract_enclosing_subgraph(G, node_pair, h)
            lg, new_index = create_line_graph_and_features(subgraph, original_features, node_pair)

            line_graphs.append((lg, new_index))


        # Save the tuple for this graph
        transformed_data.append((line_graphs, y))

    return transformed_data



In [ ]:
graphs_list = split_dataset(exp_dataset)

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [ ]:
import time
transformed_exp_datasets = []
for h in [2, 3, 5, 20, 30]:
  start_time = time.time()  # Record the start time

  transformed_exp_datasets.append(transform_graph_to_subgraphs(graphs_list, h))

  elapsed_time = time.time() - start_time  # Calculate the elapsed time
  print("Time taken to transform dataset {h}: {:.2f} seconds".format(h, elapsed_time))

Time taken to transform dataset: 4467.49 seconds


In [ ]:
transformed_dataset_5 = transformed_exp_datasets[2]

In [ ]:
import pickle
# Save the object to a file
file_path = 'transformed_dataset_5.pkl'  # Specify the path and file name

# Opening the file in binary-write mode and saving with pickle
with open(file_path, 'wb') as file:
    pickle.dump(transformed_dataset_5, file)

NameError: name 'transformed_dataset_5' is not defined

In [ ]:
# The range of different graph sizes
size_range = range(6, 16)

# The list of all graph pairs
graph_pairs = []

for n in size_range:
  for i in range(2, n//2 + 1):
    C_n = nx.cycle_graph(n)
    D_ni = nx.disjoint_union(nx.cycle_graph(n-i), nx.cycle_graph(i))
    graph_pairs.append((C_n, D_ni))
# The list of Data objects
dataset_cicles = []

# Loop over graph pairs
for pair in graph_pairs:
    # Treat each element of the pair
    for i, graph in enumerate(pair):
      graph_geom = from_networkx(graph)

      # Set the input feature x (same for all nodes)
      graph_geom.x = torch.ones((graph.number_of_nodes(), 50))

      # Set the output feature y (1 for cycle, 0 for disjoint)
      graph_geom.y = torch.tensor([1 if i == 0 else 0])

      # Add the Data object to the dataset
      dataset_cicles.append(graph_geom)

print(dataset[0], dataset[0].y)
print(dataset[0], dataset[1].y)
print(len(dataset))

In [ ]:
class RNInit(nn.Module):

    def __init__(self, random_dims = 1, distribution = 'n'):
        super().__init__()
        self.random_dims = random_dims
        self.distribution = distribution

    def forward(self, x):
        if self.random_dims > 0:

            rand = torch.empty(( x.shape[0], self.random_dims)).to(device)
            if self.distribution == "n":
                torch.nn.init.normal_(rand)
            elif self.distribution == "u":
                torch.nn.init.uniform_(rand, a=-1.0, b=1.0)
            elif self.distribution == "xn":
                torch.nn.init.xavier_normal_(rand)
            elif self.distribution == "xu":
                torch.nn.init.xavier_uniform_(rand)
            elif self.distribution == 'ones':
               torch.nn.init.ones_(rand)
            x = torch.cat([x, rand], dim=1).to(device)
        return x


##LGNN

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GraphConv, global_add_pool
from torch_geometric.data import Data

class LGNN(nn.Module):
    def __init__(self, num_layers=7, input_dim=2, hidden_dim=8, output_dim=2, rni = RNInit(), r = True):
        super(LGNN, self).__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(GraphConv(input_dim, hidden_dim, aggr='add'))  # Using add aggregation
        for _ in range(num_layers - 2):  # minus 2 because first and last layers are separately defined
            self.layers.append(GraphConv(hidden_dim, hidden_dim, aggr='add'))
        self.layers.append(GraphConv(hidden_dim, hidden_dim, aggr='add'))

        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, 32),
            nn.ELU(),
            nn.Linear(32, output_dim)
        )
        self.activation = nn.ELU()
        self.r = r
        self.rni = rni

    def forward(self, data):
        embedding_LG = []
        for line_graph, node_idx in data:
            line_graph = line_graph.to(device)

            x, edge_index = line_graph.x, line_graph.edge_index
            if self.r:
              x = self.rni(x)
            for layer in self.layers:
                x = self.activation(layer(x, edge_index))

            # Extract the embedding of the specified node
            embedding_LG.append(x[node_idx])

        # Pooling: Sum the features across the embeddings
        logit = self.pooling(embedding_LG)
        out = self.mlp(logit)

        return out

    def pooling(self, embeddings):
        # Sum the embeddings along the 0th dimension
        return torch.sum(torch.stack(embeddings), dim=0)

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()

#TRAINING

In [ ]:
from torch_geometric.loader import DataLoader  # Correct import
import torch

# Define the dataset in a way that aligns with how it is structured
# This example assumes transformed_dataset_2 is already properly defined as per your description

# Simplify the train function to focus on handling this complex structure

def train(model, loader, optimizer, device, rni = RNInit()):
    model.train()
    total_loss = 0
    total_items = 0

    for batch in loader:
        # Assuming each 'batch' is a list of tuples (graph, label)
        batch_graphs, batch_labels = zip(*batch)
        outputs = []
        optimizer.zero_grad()

        for graph, label in zip(batch_graphs, batch_labels):
           # Assuming `graph` is already a tensor or compatible type
            out = model(graph)
            outputs.append(out.unsqueeze(0))  # Unsqueeze to simulate batch dimension for stacking later

        outputs = torch.cat(outputs, dim=0)  # Concatenate along a new dimension to create a batch tensor for loss calculation
        outputs = torch.log_softmax(outputs, dim=1)
        labels = torch.tensor(batch_labels, device=device)  # Convert labels to tensor and send to device

        loss = F.nll_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_items += len(batch)

    return total_loss / total_items  # Average loss per item

def val(model, loader, device, rni = RNInit()):
    model.eval()
    total_loss = 0
    total_items = 0
    with torch.no_grad():
        outputs = []
        labels = []

        for batch in loader:
            for data, y in batch:
                data = data # Make sure data is on the correct device
                output = model(data)  # Process each item
                outputs.append(output.unsqueeze(0))  # Unsqueeze to simulate batch dimension
                labels.append(y)

        # Concatenate all outputs and labels into tensors
        outputs = torch.cat(outputs, dim=0)
        labels = torch.tensor(labels, device=device)

        # Apply log_softmax to the concatenated outputs
        outputs = torch.log_softmax(outputs, dim=1)

        # Calculate total loss using all outputs and labels
        loss = F.nll_loss(outputs, labels, reduction='sum')
        total_loss = loss.item()
        total_items = len(labels)

    return total_loss / total_items  # Return the average loss per item


def test(model, loader, device, rni = RNInit()):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch in loader:
            predictions = []
            for data, y in batch:
                output = model(data)
                pred = output.argmax(-1)
                predictions.append(pred)
            labels = torch.tensor([y for _, y in batch]).to(device)
            p = torch.tensor(predictions).to(device)
            correct += (p == labels).sum().item()
    return correct / len(loader.dataset)

In [ ]:
import random
class ManualDataLoader:
    def __init__(self, dataset, batch_size, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __iter__(self):
        self.idx = 0
        if self.shuffle:
            random.shuffle(self.dataset)
        return self

    def __next__(self):
        if self.idx >= len(self.dataset):
            raise StopIteration
        batch = self.dataset[self.idx:self.idx+self.batch_size]
        self.idx += self.batch_size
        return batch


In [ ]:
print(type(test_dataset[0][0]))

<class 'list'>


In [ ]:
def train_model(model, dataset, epochs=300, learning_rate=0.001, batch_size=20, splits=10,  modulo=4, mod_thresh=1):
    tr_accuracies = np.zeros((epochs, splits))
    tst_accuracies = np.zeros((epochs, splits))

    n = len(dataset) // splits

    for i in range(splits):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, min_lr=learning_rate)

        test_mask = torch.zeros(len(dataset), dtype=torch.bool)
        test_mask[i * n:(i + 1) * n] = 1
        learning_indices = torch.where((torch.arange(len(dataset)) % modulo <= mod_thresh) & ~test_mask)[0]
        exp_indices = torch.where((torch.arange(len(dataset)) % modulo > mod_thresh) & ~test_mask)[0]

        # Convert boolean masks to lists of indices
        test_indices = test_mask.nonzero(as_tuple=False).view(-1).tolist()
        learning_indices = learning_indices.tolist()
        exp_indices = exp_indices.tolist()

        # Now load the datasets using lists of indices
        test_dataset = [dataset[idx] for idx in test_indices]
        test_exp_dataset = [dataset[idx] for idx in exp_indices]
        test_lrn_dataset = [dataset[idx] for idx in learning_indices]
        train_dataset = [dataset[idx] for idx in range(len(dataset)) if idx not in test_indices]

        # Creating data loaders
        val_loader = ManualDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = ManualDataLoader(test_dataset, batch_size=batch_size)
        test_exp_loader = ManualDataLoader(test_exp_dataset, batch_size=batch_size)
        test_lrn_loader = ManualDataLoader(test_lrn_dataset, batch_size=batch_size)
        train_loader = ManualDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        print('---------------- Split {} ----------------'.format(i))
        best_val_loss, best_test_acc = float('inf'), 0

        for epoch in range(epochs):
            train_loss = train(model, train_loader, optimizer, device)  # Assuming train is a predefined function
            val_loss = val(model, train_loader, device)  # Using train loader for validation to simplify
            scheduler.step(val_loss)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_test_acc = test(model, test_loader, device)

            tr_accuracies[epoch, i] = test(model, train_loader, device)
            tst_accuracies[epoch, i] = best_test_acc

            print(f'Epoch: {epoch+1}, LR: {scheduler.optimizer.param_groups[0]["lr"]:.6f}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Test Acc: {best_test_acc:.4f}')

    return tr_accuracies, tst_accuracies

In [ ]:
model = LGNN(input_dim=2, num_layers=3, hidden_dim=64, output_dim=2).to(device)

In [ ]:
# Split the dataset for training with associated tuple groups maintained
results = train_model(model, dataset=transformed_dataset_20, epochs=10, learning_rate=0.001, batch_size=5, splits=5)


---------------- Split 0 ----------------
Epoch: 1, LR: 0.001000, Train Loss: 36.4952, Val Loss: 10.6937, Test Acc: 0.5000
Epoch: 2, LR: 0.001000, Train Loss: 13.2766, Val Loss: 27.6723, Test Acc: 0.5000
---------------- Split 1 ----------------
Epoch: 1, LR: 0.001000, Train Loss: 6.6842, Val Loss: 3.7899, Test Acc: 0.5000
Epoch: 2, LR: 0.001000, Train Loss: 2.2765, Val Loss: 9.9686, Test Acc: 0.5000
---------------- Split 2 ----------------
Epoch: 1, LR: 0.001000, Train Loss: 47.0610, Val Loss: 187.5068, Test Acc: 0.5000
Epoch: 2, LR: 0.001000, Train Loss: 75.0756, Val Loss: 95.0152, Test Acc: 0.5000
---------------- Split 3 ----------------
Epoch: 1, LR: 0.001000, Train Loss: 69.6269, Val Loss: 86.5442, Test Acc: 0.5000
Epoch: 2, LR: 0.001000, Train Loss: 14.8983, Val Loss: 28.4975, Test Acc: 0.5000
---------------- Split 4 ----------------
Epoch: 1, LR: 0.001000, Train Loss: 18.8354, Val Loss: 52.5636, Test Acc: 0.5000
Epoch: 2, LR: 0.001000, Train Loss: 13.4391, Val Loss: 68.6941, 

In [ ]:
model2 = LGNN(input_dim=3, num_layers=3, hidden_dim=64, output_dim=2).to(device)

In [ ]:
td_s = transformed_dataset_5[:20] + transformed_dataset_5[600:620]

In [ ]:
results = train_model(model2, dataset=td_s, epochs=30, learning_rate=0.001, batch_size=10, splits=3)

---------------- Split 0 ----------------
Epoch: 1, LR: 0.001000, Train Loss: 23.7473, Val Loss: 20.5804, Test Acc: 0.5000
Epoch: 2, LR: 0.001000, Train Loss: 9.6539, Val Loss: 19.0260, Test Acc: 0.5000
Epoch: 3, LR: 0.001000, Train Loss: 6.6140, Val Loss: 14.2363, Test Acc: 0.5000
Epoch: 4, LR: 0.001000, Train Loss: 5.3140, Val Loss: 3.1371, Test Acc: 0.5000
Epoch: 5, LR: 0.001000, Train Loss: 8.7168, Val Loss: 54.3773, Test Acc: 0.5000
Epoch: 6, LR: 0.001000, Train Loss: 5.2164, Val Loss: 25.4123, Test Acc: 0.5000
Epoch: 7, LR: 0.001000, Train Loss: 11.7806, Val Loss: 7.4605, Test Acc: 0.5000
Epoch: 8, LR: 0.001000, Train Loss: 5.4680, Val Loss: 40.0775, Test Acc: 0.5000
Epoch: 9, LR: 0.001000, Train Loss: 5.6977, Val Loss: 19.9184, Test Acc: 0.5000
Epoch: 10, LR: 0.001000, Train Loss: 7.9545, Val Loss: 8.2928, Test Acc: 0.5000
Epoch: 11, LR: 0.001000, Train Loss: 5.5633, Val Loss: 11.6710, Test Acc: 0.5000
Epoch: 12, LR: 0.001000, Train Loss: 3.9070, Val Loss: 5.3288, Test Acc: 0.50

KeyboardInterrupt: 

In [ ]:
model3 = LGNN(input_dim=22, num_layers=5, hidden_dim=64, output_dim=2, rni = RNInit(random_dims = 20)).to(device)

In [ ]:
results = train_model(model3, dataset=td_s, epochs=30, learning_rate=0.001, batch_size=10, splits=3)

---------------- Split 0 ----------------
Epoch: 1, LR: 0.001000, Train Loss: 16.3518, Val Loss: 15.8291, Test Acc: 0.5385
Epoch: 2, LR: 0.001000, Train Loss: 43.5235, Val Loss: 71.4959, Test Acc: 0.5385
Epoch: 3, LR: 0.001000, Train Loss: 19.7233, Val Loss: 80.8621, Test Acc: 0.5385
Epoch: 4, LR: 0.001000, Train Loss: 26.5086, Val Loss: 93.1845, Test Acc: 0.5385
Epoch: 5, LR: 0.001000, Train Loss: 10.3807, Val Loss: 22.1482, Test Acc: 0.5385
Epoch: 6, LR: 0.001000, Train Loss: 12.8552, Val Loss: 36.8608, Test Acc: 0.5385
Epoch: 7, LR: 0.001000, Train Loss: 5.1088, Val Loss: 45.6782, Test Acc: 0.5385
Epoch: 8, LR: 0.001000, Train Loss: 5.4612, Val Loss: 27.5692, Test Acc: 0.5385
Epoch: 9, LR: 0.001000, Train Loss: 7.1929, Val Loss: 11.1359, Test Acc: 0.5385
Epoch: 10, LR: 0.001000, Train Loss: 5.9018, Val Loss: 11.1654, Test Acc: 0.5385
Epoch: 11, LR: 0.001000, Train Loss: 4.4824, Val Loss: 25.2463, Test Acc: 0.5385
Epoch: 12, LR: 0.001000, Train Loss: 4.8207, Val Loss: 29.8293, Test A

We do not include the rest of the experiments conducted as the results are equivalent. We leave for future work how Random Node Initialisation affects the expressivity of LGNN in the context of discerning pair of graphs in the EXP dataset.
